In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor

import lightgbm as lgb
import xgboost as xgb

In [31]:
master_path = r'/Users/user/Documents/kaggle/santander/'
train_rpath = r'data/train.csv'
test_rpath = r'data/test.csv'

sample_rpath = r'data/sample_submission.csv'
output_rpath = r'output/combined.csv'

In [3]:
train_df = pd.read_csv(master_path + train_rpath)
test_df = pd.read_csv(master_path + test_rpath)

In [4]:
# prepare the data
X_train_raw = train_df.drop(["ID", "target"], axis=1)
# log since huge difference in orders
# TODO check we reverse this

y_train = np.log1p(train_df["target"].values)
# y_train = train_df["target"].values

# there is no target in test
X_test_raw = test_df.drop(["ID"], axis=1)



In [ ]:
# def summary(df):
#     n_data, n_features = df.shape[0], df.shape[1]
#     print ("Number of Records: {}".format(n_data))
#     print ("Number of Features: {}".format(n_features))
#     print(df.info())

In [ ]:
# def missingVal(df):
#     print("Total Train Features with NaN Values = " + str(df.columns[df.isnull().sum() != 0].size))
#     if (df.columns[df.isnull().sum() != 0].size):
#         print("Features with NaN => {}".format(list(df.columns[train_df.isnull().sum() != 0])))
#         df[df.columns[df.isnull().sum() != 0]].isnull().sum().sort_values(ascending=False)



In [ ]:
# # https://github.com/pandas-dev/pandas/issues/11250
# def duplicate_columns(frame):
#     groups = frame.columns.to_series().groupby(frame.dtypes).groups
#     dups = []

#     for t, v in groups.items():

#         cs = frame[v].columns
#         vs = frame[v]
#         lcs = len(cs)

#         for i in range(lcs):
#             iv = vs.iloc[:,i].tolist()
#             for j in range(i+1, lcs):
#                 jv = vs.iloc[:,j].tolist()
#                 if iv == jv:
#                     dups.append(cs[i])
#                     break

#     return dups

In [42]:
def kn_feature_selection(x_train, x_test, p_threshold):
    dropped_features = []
    for feature in x_train:
        res = stats.ks_2samp(x_train[feature], x_test[feature])
        D, p = res
#         print("D : %r,  p : %r"%(D, p))
        if p < p_threshold:
            dropped_features.append(feature)
    print(len(dropped_features))
    return x_train.drop(dropped_features, axis=1), x_test.drop(dropped_features, axis=1)

In [43]:
X_train, X_test = kn_feature_selection(X_train_raw, X_test_raw, 0.05)

1408


In [44]:
# add new features
# need to be careful so that aggregation is not used in next aggregation
def new_feat_gen(df):
    newFeat_train = pd.DataFrame()
    newFeat_train['min'] = df.min(axis=1)
    newFeat_train['max'] = df.max(axis=1)
    newFeat_train['median'] = df.median(axis=1)
    newFeat_train['q1'] = df.quantile(q=0.25, axis=1)
    newFeat_train['q3'] = df.quantile(q=0.75, axis=1)
    newFeat_train['sum'] = df.sum(axis=1)
    newFeat_train['non-zero'] = df.apply(lambda x: x.nonzero()[0].size, axis=1)
    newFeat_train['mean_non'] = newFeat_train['sum'] / newFeat_train['non-zero']
#     print(df.shape)
#     print(newFeat_train.shape)
    return pd.concat([df, newFeat_train], axis=1, ignore_index=False,sort=False)
#     return df




In [45]:
X_train = new_feat_gen(X_train)
X_test = new_feat_gen(X_test)

In [ ]:
# def topFeatures(model, X_train, y_train, title):
#     model.fit(X_train, y_train)
#     print(model)

#     feat_importances = pd.Series(model.feature_importances_, index=X_train.columns)
#     feat_importances = feat_importances.nlargest(25)
#     plt.figure(figsize=(16,8))
#     feat_importances.plot(kind='barh')
#     plt.gca().invert_yaxis()
#     plt.title(title)
#     plt.show()
#     return feat_importances

In [ ]:
# common_features_int = ['58232a6fb', '9fd594eec', '15ace8c9f', 'f190486d6', '58e2e02e6', 'eeb9cd3aa']


# # Data visualisation prep
# df_plot_pp = X_train[common_features_int]
# df_plot_pp['target'] = y_train



In [ ]:
# # common_features_union = pd.Series(list(set(s1).union(set(s2)))).values
# common_features_union = ['b43a7cfd5', '994b4c2ac', 'ced6a7e91', 'fb0f5dbfe', 'f190486d6', '58e2e02e6',
#                          '6cf7866c1', 'bc70cbc26', 'd6bb78916', 'c47340d97', '66ace2992', '58e056e12',
#                          '6eef030c1', '4af7c76b9', '2288333b4', '9fd594eec', '58232a6fb', '024c577b9',
#                          '609784003', '2a83c3267', '15ace8c9f', '402b0d650', 'd48c08bda', 'e222309b0',
#                          '20aa07010', '1702b5bf0', 'eeb9cd3aa', '1931ccfdd', '6786ea46d']



In [ ]:
# # PCA
# # why only 3, lets try 5, 10 etc
# pca = PCA(n_components=100)
# pca.fit(X_train)
# X_reduced = pca.transform(X_train)
# X_reduced_test = pca.transform(X_test)
# print(pca.explained_variance_)




In [46]:
# development and validation
# TODO use k-folds validation?
dev_X, val_X, dev_y, val_y = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)

# TODO does PCA with a tree make sense?
# TODO get book and build pipeline
# TODO cv parameters




In [24]:
# weights = {}

In [14]:
# see difference with XGboost compared to light model
# TODO check that the correct metric is being used
def run_lgb(train_X, train_y, val_X, val_y, test_X):
    params = {
        "objective": "regression",
        "metric": "rmse",
        "num_leaves": 40,
        "learning_rate": 0.005,
        "bagging_fraction": 0.7,
        "feature_fraction": 0.5,
        "bagging_frequency": 5,
        "bagging_seed": 42,
        "verbosity": -1,
        "seed": 42,

#         'boosting': 'gbdt',
#         'learning_rate': 0.01,
#         'num_leaves': 32,
#         'max_depth': 8,
#         'feature_fraction': 0.7
}
    
    
    
    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    evals_result = {}
    model = lgb.train(params, lgtrain, 5000,
                      valid_sets=[lgval],
                      early_stopping_rounds=100,
                      verbose_eval=50,
                      evals_result=evals_result)

# reverse log operation
#     pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    pred_test_y = np.expm1(model.predict(test_X, num_iteration=model.best_iteration))
    return pred_test_y, model, evals_result




In [47]:
yhat_lgbm, model_lgbm, evals_result_lgbm = run_lgb(dev_X, dev_y, val_X, val_y, X_test)
print("LightGBM Training Completed...")



Training until validation scores don't improve for 100 rounds.
[50]	valid_0's rmse: 1.61914
[100]	valid_0's rmse: 1.57077
[150]	valid_0's rmse: 1.54043
[200]	valid_0's rmse: 1.51997
[250]	valid_0's rmse: 1.50449
[300]	valid_0's rmse: 1.49475
[350]	valid_0's rmse: 1.48906
[400]	valid_0's rmse: 1.48447
[450]	valid_0's rmse: 1.48265
[500]	valid_0's rmse: 1.48148
[550]	valid_0's rmse: 1.48135
[600]	valid_0's rmse: 1.48132
Early stopping, best iteration is:
[531]	valid_0's rmse: 1.48092
LightGBM Training Completed...


In [37]:
# see difference with XGboost compared to light model
# TODO check that the correct metric is being used
def run_lgb2(train_X, train_y, val_X, val_y, test_X):
    params = {
        "objective": "regression",
        "metric": "rmse",
        "num_leaves": 30,
        "learning_rate": 0.01,
        "bagging_fraction": 0.7,
        "feature_fraction": 0.4,
        "bagging_frequency": 5,
        "bagging_seed": 42,
        "verbosity": -1,
        "seed": 42,

#         'boosting': 'gbdt',
#         'learning_rate': 0.01,
#         'num_leaves': 32,
#         'max_depth': 8,
#         'feature_fraction': 0.7
}
    
    
    
    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    evals_result = {}
    model = lgb.train(params, lgtrain, 5000,
                      valid_sets=[lgval],
                      early_stopping_rounds=100,
                      verbose_eval=50,
                      evals_result=evals_result)

# reverse log operation
#     pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    pred_test_y = np.expm1(model.predict(test_X, num_iteration=model.best_iteration))
    return pred_test_y, model, evals_result





In [48]:
yhat_lgbm2, model_lgbm2, evals_result_lgbm2 = run_lgb2(dev_X, dev_y, val_X, val_y, X_test)
print("LightGBM Training Completed2...")



Training until validation scores don't improve for 100 rounds.
[50]	valid_0's rmse: 1.57989
[100]	valid_0's rmse: 1.52856
[150]	valid_0's rmse: 1.50546
[200]	valid_0's rmse: 1.49434
[250]	valid_0's rmse: 1.48861
[300]	valid_0's rmse: 1.48568
[350]	valid_0's rmse: 1.4862
[400]	valid_0's rmse: 1.4876
Early stopping, best iteration is:
[314]	valid_0's rmse: 1.48522
LightGBM Training Completed2...


In [ ]:
# # feature importance
# print("Features Importance...")
# gain = model.feature_importance('gain')
# featureimp = pd.DataFrame({'feature':model.feature_name(),
#                    'split':model.feature_importance('split'),
#                    'gain':100 * gain / gain.sum()}).sort_values('gain', ascending=False)
# print(featureimp[:15])



In [22]:
def run_xgb(train_X, train_y, val_X, val_y, test_X):
    params = {'objective': 'reg:linear', 
      'eval_metric': 'rmse',
      'eta': 0.001,
      'max_depth': 10, 
      'subsample': 0.6, 
      'colsample_bytree': 0.6,
      'alpha':0.001,
      'random_state': 42, 
      'silent': True}
    
    xgtrain = xgb.DMatrix(train_X, train_y)
    xgvalid = xgb.DMatrix(val_X, val_y)
    
    eval_results = [(xgtrain, 'train'), (xgvalid, 'valid')]
    
    model = xgb.train(params, xgtrain, 5000, eval_results, 
                      maximize=False, 
                      verbose_eval=100, 
                      early_stopping_rounds=100)
    
    xgtest_x = xgb.DMatrix(test_X)
    x_hat_test = np.expm1(model.predict(xgtest_x, ntree_limit=model.best_ntree_limit))
    return x_hat_test, model, eval_results

In [23]:
yhat_xgb, model_xgb, eval_results_xgb = run_xgb(dev_X, dev_y, val_X, val_y, X_test)

[0]	train-rmse:14.0877	valid-rmse:14.0768
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 100 rounds.
[100]	train-rmse:12.7659	valid-rmse:12.7558
[200]	train-rmse:11.571	valid-rmse:11.5622
[300]	train-rmse:10.4915	valid-rmse:10.4837
[400]	train-rmse:9.51596	valid-rmse:9.51037
[500]	train-rmse:8.63476	valid-rmse:8.63133
[600]	train-rmse:7.83891	valid-rmse:7.83792
[700]	train-rmse:7.11998	valid-rmse:7.12164
[800]	train-rmse:6.47103	valid-rmse:6.47542
[900]	train-rmse:5.88532	valid-rmse:5.89314
[1000]	train-rmse:5.35686	valid-rmse:5.36854
[1100]	train-rmse:4.88008	valid-rmse:4.89613
[1200]	train-rmse:4.45076	valid-rmse:4.47208
[1300]	train-rmse:4.06395	valid-rmse:4.09182
[1400]	train-rmse:3.71542	valid-rmse:3.75119
[1500]	train-rmse:3.40184	valid-rmse:3.44675
[1600]	train-rmse:3.12025	valid-rmse:3.17532
[1700]	train-rmse:2.86774	valid-rmse:2.93435
[1800]	train-rmse:2.64092	valid-rmse:2.72038
[1900]	train

In [26]:
import pickle
outfile = open(r'/Users/user/Documents/kaggle/santander/xgb','wb')
pickle.dump(model_xgb,outfile)
outfile.close()

In [ ]:
# weights = {'lgbm': 1.34956, 'xgb': 1.36448}

In [49]:
submission = pd.read_csv(master_path + sample_rpath)

submission_raw = pd.DataFrame()
submission_raw["lgbm"] = yhat_lgbm
submission_raw["lgbm2"] = yhat_lgbm2
submission_raw["xgb"] = yhat_xgb
submission_raw["target"] = submission_raw.mean(axis=1)

submission["target"] = submission_raw["target"] 
# submission["target"] = (submission_raw["lgbm"] * 0.5 + submission_raw["xgb"] * 0.5)
# submission["target"] = (submission_raw["lgbm"] / weights['lgbm'] + submission_raw["xgb"]  / weights['xgb']) * (weights['lgbm'] + weights['xgb'])

In [50]:
submission_raw.head()

,lgbm,lgbm2,xgb,target
0,2.216422e+06,2.123589e+06,4531137.00,2.957049e+06
1,2.526226e+06,2.482669e+06,2878040.50,2.628979e+06
2,1.841526e+06,1.847687e+06,2571857.75,2.087024e+06
3,2.168351e+06,2.149097e+06,4510288.00,2.942579e+06
4,2.601213e+06,2.614572e+06,2777751.00,2.664512e+06


In [51]:
# save results
print(submission.head())
submission.to_csv(master_path + output_rpath, index=False)




          ID        target
0  000137c73  2.957049e+06
1  00021489f  2.628979e+06
2  0004d7953  2.087024e+06
3  00056a333  2.942579e+06
4  00056d8eb  2.664512e+06
